# Cargar librerias

In [ ]:
import torch # isntalar pytorch usando https://pytorch.org/
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar el dispositivo

Debe determinar si hay una GPU disponible y configurar su dispositivo en consecuencia.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

# Code for MPS (Apple's Metal Performance Shaders)
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# print(f"Using {device} device")


Using cuda device


# Cargar los datos de Iris

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Dividir el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir conjuntos de datos en tensores PyTorch

En PyTorch, el método .to(device) se utiliza para mover explícitamente tensores o modelos a un dispositivo específico, ya sea la CPU o una GPU. Cuando se entrenan redes neuronales, especialmente las profundas, los requerimientos computacionales pueden ser altos, y utilizar una GPU puede acelerar significativamente el proceso de entrenamiento.

In [ ]:
X_train = torch.FloatTensor(X_train).to(device)
y_train = torch.LongTensor(y_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_test = torch.LongTensor(y_test).to(device)


# Definir la estructura de la red neuronal

Crear una clase es una forma recomendada de definir modelos en PyTorch. La estructura basada en clases permite un código organizado, modular y escalable. Se pueden crear modelos más sencillos utilizando sólo funciones, pero el uso de clases proporciona una mayor flexibilidad, especialmente para arquitecturas complejas.

### Notas:
- 12 y 8 son números un tanto arbitrarios. En la práctica, la elección del número de neuronas y capas a menudo implica experimentación.
- `nn.Linear` denota capas totalmente conectadas, donde cada neurona de la capa anterior se conecta a cada neurona de la capa actual.
- ReLU (Rectified Linear Unit) es una opción popular para las capas ocultas debido a su simplicidad y eficacia.
- La última capa no suele utilizar una función de activación porque la elección de la función de pérdida en el paso siguiente (criterio) a veces la incluye.
  - Para tareas de clasificación con múltiples clases, `CrossEntropyLoss` en PyTorch combina una activación SoftMax con una pérdida de entropía cruzada.


In [ ]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 12)    # Primera capa oculta con 12 neuronas
        self.fc2 = nn.Linear(12, 8)   # Segunda capa oculta con 8 neuronas
        self.fc3 = nn.Linear(8, 3)    # Capa de salida con 3 neuronas (para las 3 clases)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Aplicar la función de activación ReLU después de la primera capa oculta
        x = torch.relu(self.fc2(x))  # Aplicar la función de activación ReLU después de la segunda capa oculta
        x = self.fc3(x)              # No hay activación aquí ya que usaremos CrossEntropyLoss
        return x

In [ ]:
model = IrisNet().to(device)

# Definir la función de pérdida (error) y el optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()               # This combines a SoftMax activation and a cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam optimizer with learning rate of 0.01

# Bucle de entrenamiento

In [ ]:
best_val_loss = float('inf')  # Comenzar con una pérdida inicial muy alta
patience = 10  # Definir cuántas épocas esperar sin mejora
counter = 0  # Inicializar el contador

for epoch in range(500):   # Aumentar las épocas para asegurar la convergencia con datos sin procesar
    optimizer.zero_grad()  # Limpiar los gradientes del último paso
    out = model(X_train)   # Paso hacia adelante: calcular la y predicha pasando x al modelo
    loss = criterion(out, y_train) # Calcular la pérdida
    loss.backward()        # Paso hacia atrás: calcular el gradiente de la pérdida con respecto a los parámetros del modelo
    optimizer.step()       # Actualizar los parámetros del modelo

    # Evaluar el rendimiento del modelo en los datos de validación
    # Asegurarse de que no se calculen gradientes durante este paso para ahorrar computación y memoria
    with torch.no_grad():
        val_out = model(X_test) # Pasar los datos de validación por el modelo para obtener predicciones
        val_loss = criterion(val_out, y_test) # Calcular la pérdida de validación basada en las predicciones del modelo y las etiquetas reales de los datos de validación

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0  # Reiniciar el contador ya que hemos observado una mejora en la pérdida de validación
    else:
        counter += 1  # Si la pérdida de validación no mejora, incrementar el contador

    # Si el número de épocas sin mejora excede nuestra paciencia establecida, detener el entrenamiento
    if counter >= patience:
        print("Detención temprana debido a falta de mejora!")
        break  # Salir del bucle de entrenamiento

    if (epoch+1) % 50 == 0:  # Imprimir la pérdida cada 50 épocas
        print(f"Época {epoch+1}, Pérdida: {loss.item()}")

Epoch 50, Loss: 0.7173503637313843
Epoch 100, Loss: 0.5290992856025696
Epoch 150, Loss: 0.4140053987503052
Epoch 200, Loss: 0.34220272302627563
Epoch 250, Loss: 0.2906952500343323
Epoch 300, Loss: 0.2525676190853119
Epoch 350, Loss: 0.22402741014957428
Epoch 400, Loss: 0.20210480690002441
Epoch 450, Loss: 0.18465901911258698
Epoch 500, Loss: 0.17038440704345703


# Evaluar el modelo

In [ ]:
with torch.no_grad(): # Desactivar el cálculo de gradientes durante la evaluación para ahorrar memoria y acelerar el proceso
    test_out = model(X_test)  # Paso hacia adelante: calcular las salidas predichas pasando los datos de prueba al modelo
    _, predicted = torch.max(test_out, 1) # Obtener las etiquetas de clase con las probabilidades predichas más altas
    accuracy = accuracy_score(y_test.cpu().numpy(), predicted.cpu().numpy()) # Calcular la precisión comparando las etiquetas predichas con las etiquetas verdaderas
    print(f"Precisión en prueba: {accuracy * 100:.2f}%")

Test Accuracy: 100.00%


# Despliegue
Para desplegar un modelo PyTorch


In [ ]:
# Guardar el modelo
torch.save(model.state_dict(), "iris_model.pth")

In [ ]:
# Cargar el modelo para inferencia
model = IrisNet().to(device)
model.load_state_dict(torch.load("iris_model.pth"))
model.eval()  # Configurar el modelo en modo de evaluación

IrisNet(
  (fc1): Linear(in_features=4, out_features=12, bias=True)
  (fc2): Linear(in_features=12, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
)

In [ ]:
# Hacer una predicción con datos nuevos

# Supongamos que tienes nuevos datos para predicción en forma de un array de numpy
new_data = [[5.1, 3.5, 1.4, 0.2],  # Algunas medidas de iris
            [6.7, 3.0, 5.2, 2.3]]  # Otro conjunto de medidas de iris

# Convertir los datos a un tensor de PyTorch
input_tensor = torch.FloatTensor(new_data)

# Si usaste una GPU durante el entrenamiento, mueve el tensor de entrada al mismo dispositivo
if torch.cuda.is_available():
    input_tensor = input_tensor.to('cuda')

# Obtener las predicciones del modelo
with torch.no_grad():  # Esto asegura que la operación no sea seguida por autograd de PyTorch
    outputs = model(input_tensor)

# Obtener las clases predichas
_, predicted_classes = torch.max(outputs, 1)

# Convertir las clases predichas a una lista
predicted_classes = predicted_classes.tolist()

print(predicted_classes)  # Esto te dará los índices de las clases predichas para cada entrada

[0, 2]


# Más

Otras cosas que deberían ser parte de un modelo de aprendizaje profundo

- Ajust de hiperparámetros
- Tasa de aprendizaje
- Tamaño de lote
- Épocas
- Optimizador
- Arquitectura de red
- Abandono
- Regularización
- Momento
